Note: based  on “Recognize Flowers with TensorFlow on Android” TensorFlow tutorial.

In [3]:
import tensorflow as tf

import os
import numpy as np
import matplotlib.pyplot as plt

In [4]:

base_dir = "top_landmarks"

The following code loads the dataset as images and breaks into a training and a validation batch.

In [5]:
IMAGE_SIZE = 224
BATCH_SIZE = 64

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2)

train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='training')

val_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='validation')

Found 2039 images belonging to 9 classes.
Found 505 images belonging to 9 classes.


In [6]:
for image_batch, label_batch in train_generator:
  break
image_batch.shape, label_batch.shape

((64, 224, 224, 3), (64, 9))

The coin labels are saved in "labels.txt"

In [7]:
print (train_generator.class_indices)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
  f.write(labels)

{'bled_island': 0, 'eiffel_tower': 1, 'golden_gate_bridge': 2, 'grand_canyon': 3, 'machu_picchu': 4, 'niagara_falls': 5, 'presern_square': 6, 'stonehenge': 7, 'sydney_opera_house': 8}


In [8]:
!cat labels.txt

'cat' is not recognized as an internal or external command,
operable program or batch file.


## Create the base model from a pre-trained network
Create the base model from the MobileNet V2 model developed at Google, and pre-trained on the ImageNet dataset, a large dataset of 1.4M images and 1000 classes of web images.

First, pick which intermediate layer of MobileNet V2 will be used for feature extraction. A common practice is to use the output of the very last layer before the flatten operation, the so-called "bottleneck layer". The reasoning here is that the following fully-connected layers will be too specialized to the task the network was trained on, and thus the features learned by these layers won't be very useful for a new task. The bottleneck features, however, retain much generality.

Let's instantiate an MobileNet V2 model pre-loaded with weights trained on ImageNet. By specifying the include_top=False argument, we load a network that doesn't include the classification layers at the top, which is ideal for feature extraction.

In [8]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                              include_top=False, 
                                              weights='imagenet')

## Feature extraction

You will freeze the convolutional base created from the previous step and use that as a feature extractor, add a classifier on top of it and train the top-level classifier.


In [9]:
base_model.trainable = False

## Add a classification head

In [12]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(9, activation='softmax')
])

## Compile the model

In [13]:
model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [14]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 conv2d_2 (Conv2D)           (None, 5, 5, 32)          368672    
                                                                 
 dropout_2 (Dropout)         (None, 5, 5, 32)          0         
                                                                 
 global_average_pooling2d_2  (None, 32)                0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_2 (Dense)             (None, 9)                 297       
                                                                 
Total params: 2626953 (10.02 MB)
Trainable params: 368

We added a convolution layer and a dense layer that can be trained. Here is the list of all the variables that will be trained (note that these are matrices).

In [15]:
print('Number of trainable variables = {}'.format(len(model.trainable_variables)))
variables_names = [v.name for v in model.trainable_variables]
for k in variables_names:
    print("Variable: ", k)

Number of trainable variables = 4
Variable:  conv2d_2/kernel:0
Variable:  conv2d_2/bias:0
Variable:  dense_2/kernel:0
Variable:  dense_2/bias:0


## Train the model
Ideally, you would like to train your model with, say, 10 epoch. However, if this takes prohibitively long time on your laptop, for the exercise sake use only one epoch.

In [16]:
epochs = 10 # ideally 10

history = model.fit(train_generator, epochs=epochs, validation_data=val_generator)

Epoch 1/10
32/32 [==============================] - 128s 4s/step - loss: 1.3449 - accuracy: 0.5866 - val_loss: 0.6289 - val_accuracy: 0.8376
Epoch 2/10
32/32 [==============================] - 94s 3s/step - loss: 0.3255 - accuracy: 0.9039 - val_loss: 0.3270 - val_accuracy: 0.9208
Epoch 3/10
32/32 [==============================] - 92s 3s/step - loss: 0.1052 - accuracy: 0.9745 - val_loss: 0.2901 - val_accuracy: 0.9149
Epoch 4/10
32/32 [==============================] - 89s 3s/step - loss: 0.0563 - accuracy: 0.9902 - val_loss: 0.2983 - val_accuracy: 0.9228
Epoch 5/10
32/32 [==============================] - 88s 3s/step - loss: 0.0331 - accuracy: 0.9961 - val_loss: 0.2763 - val_accuracy: 0.9327
Epoch 6/10
32/32 [==============================] - 88s 3s/step - loss: 0.0190 - accuracy: 0.9995 - val_loss: 0.2626 - val_accuracy: 0.9366
Epoch 7/10
32/32 [==============================] - 89s 3s/step - loss: 0.0143 - accuracy: 1.0000 - val_loss: 0.2660 - val_accuracy: 0.9426
Epoch 8/10
32/32 [=

## Save the model to a file

In [17]:
saved_model_dir = 'saved_models' # Put the name of the directory where you want to save the model
tf.saved_model.save(model, saved_model_dir)

#converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
#tflite_model = converter.convert()

#with open('model.tflite', 'wb') as f:
#  f.write(tflite_model)

INFO:tensorflow:Assets written to: saved_models\assets


INFO:tensorflow:Assets written to: saved_models\assets


## Converting the model from TensorFlow to TensorFlow Lite

In [18]:
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

In [19]:
with open('lookitecture_model.tflite', 'wb') as f:
  f.write(tflite_model)

Note: If the above code does not run without errors with your version of TF and Python, please use "tflite_convert --saved_model_dir=YOUR_MODEL_DIR --output_file=mobilenet.tflite" from the command line.

## Adding metadata to TensorFlow Lite models 
TensorFlow Lite metadata provides a standard for model descriptions. The metadata is an important source of knowledge about what the model does and its input / output information. The metadata consists of both

human readable parts which convey the best practice when using the model, and
machine readable parts that can be leveraged by code generators, such as the TensorFlow Lite Android code generator and the Android Studio ML Binding feature.

You can find more details about TF Lite model metadata here: https://www.tensorflow.org/lite/models/convert/metadata

In [20]:
from tflite_support import metadata_schema_py_generated as _metadata_fb
from tflite_support import metadata as _metadata
import flatbuffers

In [21]:
class ModelSpecificInfo(object):
  """Holds information that is specificly tied to an image classifier."""

  def __init__(self, name, version, image_width, image_height, image_min,
               image_max, mean, std, num_classes, author):
    self.name = name
    self.version = version
    self.image_width = image_width
    self.image_height = image_height
    self.image_min = image_min
    self.image_max = image_max
    self.mean = mean
    self.std = std
    self.num_classes = num_classes
    self.author = author

_MODEL_INFO = {
    "lookitecture_model.tflite":
        ModelSpecificInfo(
            name="MobileNetV2",
            version="v1",
            image_width=224,
            image_height=224,
            image_min=0,
            image_max=255,
            mean=[127.5],
            std=[127.5],
            num_classes=9,
            author="TensorFlow")
}

In [22]:
class MetadataPopulatorForImageClassifier(object):
  """Populates the metadata for an image classifier."""

  def __init__(self, model_file, model_info, label_file_path):
    self.model_file = model_file
    self.model_info = model_info
    self.label_file_path = label_file_path
    self.metadata_buf = None

  def populate(self):
    """Creates metadata and then populates it for an image classifier."""
    self._create_metadata()
    self._populate_metadata()

  def _create_metadata(self):
    """Creates the metadata for an image classifier."""

    # Creates model info.
    model_meta = _metadata_fb.ModelMetadataT()
    model_meta.name = self.model_info.name
    model_meta.description = ("Identify the most prominent object in the "
                              "image from a set of %d categories." %
                              self.model_info.num_classes)
    model_meta.version = self.model_info.version
    model_meta.author = self.model_info.author
    model_meta.license = ("Apache License. Version 2.0 "
                          "http://www.apache.org/licenses/LICENSE-2.0.")

    # Creates input info.
    input_meta = _metadata_fb.TensorMetadataT()
    input_meta.name = "image"
    input_meta.description = (
        "Input image to be classified. The expected image is {0} x {1}, with "
        "three channels (red, blue, and green) per pixel. Each value in the "
        "tensor is a single byte between {2} and {3}.".format(
            self.model_info.image_width, self.model_info.image_height,
            self.model_info.image_min, self.model_info.image_max))
    input_meta.content = _metadata_fb.ContentT()
    input_meta.content.contentProperties = _metadata_fb.ImagePropertiesT()
    input_meta.content.contentProperties.colorSpace = (
        _metadata_fb.ColorSpaceType.RGB)
    input_meta.content.contentPropertiesType = (
        _metadata_fb.ContentProperties.ImageProperties)
    input_normalization = _metadata_fb.ProcessUnitT()
    input_normalization.optionsType = (
        _metadata_fb.ProcessUnitOptions.NormalizationOptions)
    input_normalization.options = _metadata_fb.NormalizationOptionsT()
    input_normalization.options.mean = self.model_info.mean
    input_normalization.options.std = self.model_info.std
    input_meta.processUnits = [input_normalization]
    input_stats = _metadata_fb.StatsT()
    input_stats.max = [self.model_info.image_max]
    input_stats.min = [self.model_info.image_min]
    input_meta.stats = input_stats

    # Creates output info.
    output_meta = _metadata_fb.TensorMetadataT()
    output_meta.name = "probability"
    output_meta.description = "Probabilities of the %d labels respectively." % self.model_info.num_classes
    output_meta.content = _metadata_fb.ContentT()
    output_meta.content.content_properties = _metadata_fb.FeaturePropertiesT()
    output_meta.content.contentPropertiesType = (
        _metadata_fb.ContentProperties.FeatureProperties)
    output_stats = _metadata_fb.StatsT()
    output_stats.max = [1.0]
    output_stats.min = [0.0]
    output_meta.stats = output_stats
    label_file = _metadata_fb.AssociatedFileT()
    label_file.name = os.path.basename(self.label_file_path)
    label_file.description = "Labels for objects that the model can recognize."
    label_file.type = _metadata_fb.AssociatedFileType.TENSOR_AXIS_LABELS
    output_meta.associatedFiles = [label_file]

    # Creates subgraph info.
    subgraph = _metadata_fb.SubGraphMetadataT()
    subgraph.inputTensorMetadata = [input_meta]
    subgraph.outputTensorMetadata = [output_meta]
    model_meta.subgraphMetadata = [subgraph]

    b = flatbuffers.Builder(0)
    b.Finish(
        model_meta.Pack(b),
        _metadata.MetadataPopulator.METADATA_FILE_IDENTIFIER)
    self.metadata_buf = b.Output()

  def _populate_metadata(self):
    """Populates metadata and label file to the model file."""
    populator = _metadata.MetadataPopulator.with_model_file(self.model_file)
    populator.load_metadata_buffer(self.metadata_buf)
    populator.load_associated_files([self.label_file_path])
    populator.populate()


In [ ]:
model_file = "lookitecture_model.tflite"
model_basename = os.path.basename(model_file)

export_model_path = os.path.join(saved_model_dir, model_basename)

tf.io.gfile.copy(model_file, export_model_path, overwrite=True)

populator = MetadataPopulatorForImageClassifier(
  export_model_path, _MODEL_INFO.get(model_file), "labels.txt")
populator.populate()

In [ ]:

displayer = _metadata.MetadataDisplayer.with_model_file(export_model_path)
export_json_file = os.path.join(saved_model_dir, os.path.splitext(model_basename)[0] + ".json")
json_file = displayer.get_metadata_json()
with open(export_json_file, "w") as f:
    f.write(json_file)

In [25]:
print("Finished populating metadata and associated file to the model:")
print(model_file)
print("The metadata json file has been saved to:")
print(export_json_file)
print("The associated file that has been been packed to the model is:")
print(displayer.get_packed_associated_file_list())

Finished populating metadata and associated file to the model:
lookitecture_model.tflite
The metadata json file has been saved to:
saved_models\lookitecture_model.json
The associated file that has been been packed to the model is:
['labels.txt']


You are done creating the model! The model file you want to use in your Android app is stored in the "saved_models" folder. Go back to the lab instructions to see how to load it in your new app.